## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
print(application_df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9','T13','T12','T2','T25','T14','T29','T15','T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
print(application_df['CLASSIFICATION'].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1248        1
C6100        1
C1820        1
C1900        1
C2150        1
Name: count, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
print(application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 1])

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C1720        6
C2400        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1256        2
C1246        2
C1234        2
C1267        2
Name: count, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 1000].index.tolist()


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [9]:
# Convert categorical data to numeric with `pd.get_dummies`

categorical_cols = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS']
application_df_encoded = pd.get_dummies(application_df, columns=categorical_cols, drop_first=True)


income_mapping = {
    '0': 0,
    '1-9999': 5000,
    '10000-24999': 17500,
    '25000-99999': 62500,
    '100000-499999': 300000,
    '500000-999999': 750000,
    '1M-5M': 3000000,
    '5M-10M': 7500000,
    '10M-50M': 30000000,
    '50M+': 50000000
}

# Apply the mapping to convert INCOME_AMT to numerical values
application_df_encoded['INCOME_AMT'] = application_df_encoded['INCOME_AMT'].map(income_mapping)

application_df_encoded.head()


,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_Y
0,1,0,5000,1,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,1,5000,108590,1,False,False,True,False,False,False,...,False,False,False,False,True,False,True,False,False,False
2,1,0,5000,0,False,False,False,False,True,False,...,True,False,False,False,False,True,False,False,False,False
3,1,17500,6692,1,False,False,True,False,False,False,...,False,False,False,False,True,False,False,False,True,False
4,1,300000,142590,1,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop(columns=['IS_SUCCESSFUL'])
y = application_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

1.   Elemento de la lista
2.   Elemento de la lista



In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=input_features))
nn.add(tf.keras.layers.BatchNormalization())
nn.add(tf.keras.layers.Dropout(0.2))  # Dropout to prevent overfitting

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))
nn.add(tf.keras.layers.BatchNormalization())
nn.add(tf.keras.layers.Dropout(0.2))



# Third hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation='relu'))
nn.add(tf.keras.layers.BatchNormalization())



# Output layer with 1 neuron (sigmoid for binary classification)
#nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))



# Output layer
#  YOUR CODE GOES HERE

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 128)                 │           3,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,105 (59.00 KB)

 Trainable params: 14,657 (57.25 KB)

 Non-trainable params: 448 (1.75 KB)

In [19]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=50, validation_split=0.2, verbose=1)


Epoch 1/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6773 - loss: 0.6330 - val_accuracy: 0.7318 - val_loss: 0.5633
Epoch 2/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.7093 - loss: 0.5851 - val_accuracy: 0.7341 - val_loss: 0.5554
Epoch 3/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7205 - loss: 0.5717 - val_accuracy: 0.7341 - val_loss: 0.5556
Epoch 4/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7214 - loss: 0.5691 - val_accuracy: 0.7363 - val_loss: 0.5566
Epoch 5/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7194 - loss: 0.5694 - val_accuracy: 0.7385 - val_loss: 0.5527
Epoch 6/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7281 - loss: 0.5574 - val_accuracy: 0.7380 - val_loss: 0.5562
Epoch 7/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7243 - loss: 0.5630 - val_accuracy: 0.7396 - val_loss: 0.5509
Epoch 8/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7311 - loss: 0.5600 - val_accuracy: 0.

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.7281 - loss: 0.5597
Loss: 0.5596897602081299, Accuracy: 0.7281340956687927


In [23]:
# Train the model for the second time with different values
history = nn.fit(X_train_scaled, y_train, epochs=200, validation_split=0.2, batch_size=32,verbose=1)


Epoch 1/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7273 - loss: 0.5572 - val_accuracy: 0.7391 - val_loss: 0.5464
Epoch 2/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7236 - loss: 0.5575 - val_accuracy: 0.7396 - val_loss: 0.5444
Epoch 3/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7270 - loss: 0.5560 - val_accuracy: 0.7383 - val_loss: 0.5459
Epoch 4/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7275 - loss: 0.5545 - val_accuracy: 0.7372 - val_loss: 0.5466
Epoch 5/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7269 - loss: 0.5574 - val_accuracy: 0.7414 - val_loss: 0.5439
Epoch 6/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.7316 - loss: 0.5542 - val_accuracy: 0.7392 - val_loss: 0.5439
Epoch 7/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7302 - loss: 0.5546 - val_accuracy: 0.7382 - val_loss: 0.5456
Epoch 8/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7303 - loss: 0.5495 - val_accu

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


215/215 - 0s - 2ms/step - accuracy: 0.7258 - loss: 0.5607
Loss: 0.5607227683067322, Accuracy: 0.7258017659187317


In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=input_features))
nn.add(tf.keras.layers.BatchNormalization())
nn.add(tf.keras.layers.Dropout(0.2))  # Dropout to prevent overfitting

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))
nn.add(tf.keras.layers.BatchNormalization())
nn.add(tf.keras.layers.Dropout(0.2))



# Third hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))
nn.add(tf.keras.layers.BatchNormalization())

# fourh hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='sigmoid'))
nn.add(tf.keras.layers.BatchNormalization())



# Output layer with 1 neuron (sigmoid for binary classification)
#nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))



# Output layer
#  YOUR CODE GOES HERE

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 128)                 │           3,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,761 (85.00 KB)

 Trainable params: 21,121 (82.50 KB)

 Non-trainable params: 640 (2.50 KB)

In [26]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=200, validation_split=0.2, batch_size=32,verbose=1)


Epoch 1/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.6729 - loss: 0.6334 - val_accuracy: 0.7372 - val_loss: 0.5596
Epoch 2/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7150 - loss: 0.5806 - val_accuracy: 0.7349 - val_loss: 0.5557
Epoch 3/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7175 - loss: 0.5725 - val_accuracy: 0.7362 - val_loss: 0.5681
Epoch 4/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7144 - loss: 0.5763 - val_accuracy: 0.7385 - val_loss: 0.5536
Epoch 5/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7217 - loss: 0.5662 - val_accuracy: 0.7358 - val_loss: 0.5581
Epoch 6/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.7295 - loss: 0.5638 - val_accuracy: 0.7358 - val_loss: 0.5498
Epoch 7/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7252 - loss: 0.5654 - val_accuracy: 0.7352 - val_loss: 0.5514
Epoch 8/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7267 - loss: 0.5660 - val_accu

In [28]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.7245 - loss: 0.5607
Loss: 0.560696542263031, Accuracy: 0.7244898080825806


In [28]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")


NameError: name 'nn' is not defined